In [3]:
#new jupyter notebook

from tqdm import tqdm
from dataset import get_data_loader, move_data_to_device
from hparams import Hparams

train_loader = get_data_loader(split='train', args=Hparams.args)
for data in tqdm(train_loader):
    mel_spectrogram, yin, pyin = data#move_data_to_device(data, 'cpu')
    print(mel_spectrogram.shape)
    print(yin.shape)
    print(pyin.shape)
    #assert list(x.shape) == [8, 250, 256]  # shape in [B, T, D],
                                # i.e., [Batch size, num of frame per sample, spectrogram feature dimension]
    #assert list(onset.shape) == list(offset.shape) == list(octave.shape) == list(pitch_class.shape) == [8, 250]
    break
print('Congrats!')

  0%|          | 0/10 [00:13<?, ?it/s]

torch.Size([8, 256, 5])
torch.Size([8, 256])
torch.Size([8, 256])
Congrats!


In [ ]:
#For the Michigan dataset

from tqdm import tqdm
from dataset import  get_data_loader_michigan
from hparams import Hparams_michigan

train_ds, test_ds , data_loader_train, data_loader_test = get_data_loader_michigan(args=Hparams_michigan.args, test_size=0.2)

# to plot
train_ds.plot_item(0)

# data loading
for data in tqdm(data_loader_train):
    mel_spectrogram_normalised_log_scale_torch, yin_normalised_torch, pyin_normalised_torch, word, toneclass = data
    print(f"(Batch, feature)")
    print(f"Spectrogram: {mel_spectrogram_normalised_log_scale_torch.shape} {type(mel_spectrogram_normalised_log_scale_torch)}")
    print(f"Yin: {yin_normalised_torch.shape} {type(yin_normalised_torch)}")
    print(f"Pyin: {pyin_normalised_torch.shape} {type(pyin_normalised_torch)}")
    print(f"Word: {len(word)} {type(word)}")
    print(f"Toneclass: {toneclass.shape} {type(toneclass)}")
    break
print('Congrats!')

In [ ]:
import torch

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

class Metrics:
    def __init__(self):
        self.buffer = {}

    def update(self, out, tgt, loss):
        with torch.no_grad():
            out = torch.flatten(out)
            tgt = torch.flatten(tgt)

            # metric computation
            f1 = f1_score(tgt.cpu(), out.cpu())
            # acc = accuracy_score(tgt.cpu(), out.cpu())

            batch_metric = {
                'loss': loss.item(),
                'f1_score': f1,
            }

            for k in batch_metric:
                if k in self.buffer:
                    self.buffer[k].append(batch_metric[k])
                else:
                    self.buffer[k] = [batch_metric[k]]

    def get_value(self):
        for k in self.buffer:
            self.buffer[k] = sum(self.buffer[k]) / len(self.buffer[k])
        ret = self.buffer
        self.buffer = {}
        return ret

In [ ]:
import os
import time

import torch
import torch.nn as nn
import torch.optim as optim

from model import ToneEval_Base


def fit(args, learning_params):
    # Set paths
    save_model_dir = f"{args['save_model_dir']}/{learning_params['lr']}"
    if not os.path.exists(save_model_dir):
        os.mkdir(save_model_dir)

    model = ToneEval_Base(input_shape=(1, 1))
    optimizer = optim.AdamW(model.parameters(), lr=learning_params['lr'])
    loss_func = nn.CrossEntropyLoss()
    metric = Metrics()

    # Start training
    print('Start training...')
    start_time = time.time()
    best_model_id = -1
    min_valid_loss = 10000

    for epoch in range(1, learning_params['epoch'] + 1):
        model.train()
        total_training_loss = 0

        # Train
        pbar = tqdm(data_loader_train)
        for batch_idx, batch in enumerate(pbar):
            x, tone_class = move_data_to_device(batch, args['device'])
            out = model(x)
            loss = loss_func(out, tone_class)
            metric.update(out, tone_class, loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_training_loss += loss.item()

            pbar.set_description('Epoch {}, Loss: {:.4f}'.format(epoch, loss.item()))
        metric_train = metric.get_value()

        # Validation
        model.eval()
        with torch.no_grad():
            for batch_idx, batch in enumerate(data_loader_test):
                x, tone_class = move_data_to_device(batch, args['device'])
                out = model(x)
                metric.update(out, tone_class)
        metric_test = metric.get_value()

        # Logging
        print('[Epoch {:02d}], Train Loss: {:.5f}, Valid Loss {:.5f}, Time {:.2f}s'.format(
            epoch, metric_train['loss'], metric_test['loss'], time.time() - start_time,
        ))
        print('Split Train Loss, F1: Loss {:.4f} | F1_Score {:.4f}'.format(
            metric_train['loss'],
            metric_train['f1_score']
        ))
        print('Split Test Loss, F1: Loss {:.4f} | F1_Score {:.4f}'.format(
            metric_test['loss'],
            metric_test['f1_score']
        ))

        # Save the best model
        if metric_test['loss'] < min_valid_loss:
            min_valid_loss = metric_test['loss']
            best_model_id = epoch

            save_dict = model.state_dict()
            target_model_path = save_model_dir + '/best_model.pth'
            torch.save(save_dict, target_model_path)

    print('Training done in {:.1f} minutes.'.format((time.time() - start_time) / 60))
    return best_model_id

In [ ]:
# Set learning params
learning_params = {
    'batch_size': 50,
    'epoch': 10,
    'lr': 1e-3,
}

fit(args=Hparams.args, learning_params=learning_params)